# Practice Session 04: Basket analysis

Author: <font color="blue">José M. Pérez Clar</font>

E-mail: <font color="blue">josemaria.perez02@estudiant.upf.edu</font>

Date: <font color="blue">18/10/2023</font>

In [ ]:
# instacart-transactions.csv.gz available at https://drive.google.com/file/d/1KWwxNeIgB2A1wt7_TOb8jxa5m4ezkUWd/view?usp=sharing

In [8]:
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd  
import csv
import gzip
from apyori import apriori

# 1. Playing with apyori

In [9]:
# LEAVE AS-IS

def print_apyori_output (association_results, info=False, info_key=False):
    for relation_record in association_results:
        itemset = list(relation_record.items)
        
        # Consider only itemsets of two elements
        if len(itemset) > 1: 
        
            print("Rules involving itemset %s" % itemset)
            support = relation_record.support

            for rules in relation_record.ordered_statistics:
                antecedent = list(rules.items_base)
                consequent = list(rules.items_add)
                
                if info_key:
                    antecedent = [info.loc[x][info_key] for x in antecedent]
                    consequent = [info.loc[x][info_key] for x in consequent]
                
                confidence = rules.confidence
                lift = rules.lift

                print("%s => %s (support=%.4f, confidence=%.2f, lift=%.2f)" %
                      (antecedent, consequent, support, confidence, lift))
            print()

In [28]:
transactions = [
    ['beer', 'chips', 'nuts'],
    ['beer', 'olives'],
    ['chips', 'olives'],
    ['beer', 'chips', 'nuts', 'olives'],
    ['nuts', 'olives'],
    ['beer', 'chips'],
    ['beer', 'nuts'],
    ['chips', 'nuts', 'olives'],
    ['beer', 'chips', 'nuts', 'olives'],
    ['chips', 'nuts'],
    ['beer', 'olives'],
    ['beer', 'chips', 'nuts'],
    ['beer', 'chips', 'olives'],
    ['nuts', 'olives'],
    ['beer', 'chips', 'nuts'],
    ['chips', 'nuts'],
    ['chips', 'olives'],
    ['beer', 'chips' , 'nuts'],
    ['nuts'],
    ['beer', 'chips', 'nuts', 'olives'],
]

results = list(apriori(transactions, min_support=0.45, min_confidence=0.71, min_lift=1.0))
print_apyori_output(results)

Rules involving itemset ['beer', 'chips']
['beer'] => ['chips'] (support=0.4500, confidence=0.75, lift=1.07)

Rules involving itemset ['nuts', 'chips']
['chips'] => ['nuts'] (support=0.5000, confidence=0.71, lift=1.02)
['nuts'] => ['chips'] (support=0.5000, confidence=0.71, lift=1.02)




1. Rule: ['beer'] => ['chips']
    - Support: Support is the proportion of transactions that contain both 'beer' and 'chips'. It is calculated as the number of transactions containing both items divided by the total number of transactions. In this case, support = 9 / 20 = 0.45.

    - Confidence: Confidence is the conditional probability that a transaction containing 'beer' also contains 'chips'. It is calculated as the support of the itemset ['beer', 'chips'] divided by the support of the itemset ['beer']. In this case, confidence = support(['beer', 'chips']) / support(['beer']) = 0.45 / 0.6 = 0.75.

    - Lift: Lift measures the degree of association between 'beer' and 'chips' while taking into account the overall prob of these items. It is calculated as (support(['beer', 'chips']) / (support(['beer']) * support(['chips'])). In this case, lift = (0.45 / (0.6 * 0.6)) = 1.07.


2. Rule: ['chips'] => ['nuts']
3. Rule: ['nuts'] => ['chips']
    - Support: Support is the proportion of transactions that contain both 'nuts' and 'chips'. It is calculated as the number of transactions containing both items divided by the total number of transactions. In this case, support = 10 / 20 = 0.5.

    - Confidence: Confidence is the conditional probability that a transaction containing 'chips' also contains 'nuts' and viceversa. It is calculated as the support of the itemset ['chips', 'nuts'] divided by the support of the itemset ['chips'] or ['nuts']. In both cases, confidence = support(['chips', 'nuts']) / support(['chips']) = 0.5 / 0.7 = 0.71.

    - Lift: Lift measures the degree of association between 'chips' and 'nuts' while taking into account the overall prob of these items. It is calculated as (support(['chips', 'nuts']) / (support(['chips']) * support(['nuts'])). In both cases, lift = (0.5 / (0.7 * 0.7)) = 1.02.

# 2. Load and prepare the shopping baskets

In [11]:
# LEAVE AS-IS

# File names
INPUT_PRODUCTS = "instacart-products.csv"
INPUT_TRANSACTIONS = "instacart-transactions.csv.gz"

# Read into a dataframe
products = pd.read_csv(INPUT_PRODUCTS, delimiter=",")

# Set product_id as index, and drop column aisle_id
products = products.set_index('product_id').drop(columns=['aisle_id'])

products.head(100)

,product_name,department_id
product_id,,
1,Chocolate Sandwich Cookies,19
2,All-Seasons Salt,13
3,Robust Golden Unsweetened Oolong Tea,7
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,1
5,Green Chile Anytime Sauce,13
...,...,...
96,Sprinklez Confetti Fun Organic Toppings,13
97,Organic Chamomile Lemon Tea,7
98,2% Yellow American Cheese,16


## 2.1. Select by department

In [12]:
# LEAVE AS-IS

DEPT_BAKERY = 3
DEPT_VEGGIES = 4
DEPT_ALCOHOL = 5
DEPT_WORLD = 6
DEPT_DRINKS = 7
DEPT_PETS = 8
DEPT_PHARMACY = 11
DEPT_CLEANING = 17
DEPT_BABIES = 18

In [13]:
def select_from_departments(df , prod_ids, department_ids):
    
    return_list = []
    
    print("Input Products:")
    for product_id in prod_ids:
        dep_id = df.loc[product_id].department_id
        prod_name = df.loc[product_id].product_name
        print(product_id, prod_name, ", department number:" ,dep_id )
        
        if dep_id in department_ids:
            return_list.append(product_id)
            
            
    print("\nSelected Products:")
    for product_id in return_list:
        dep_id = df.loc[product_id].department_id
        prod_name = df.loc[product_id].product_name
        print(product_id, prod_name, ", department number:" ,dep_id )

In [14]:
#TEST0
print("\n------------------------------------------------------------------------------------------")
print("EXAMPLE\n")
product_id = [22, 26, 45, 54, 57, 71, 111, 112]
dep_id = [DEPT_BAKERY, DEPT_CLEANING]
select_from_departments(products , product_id, dep_id)

#TEST1
print("\n------------------------------------------------------------------------------------------")
print("TEST CASE 1\n")
product_id = [42, 15, 91, 47, 63, 76, 32, 14]
dep_id = [DEPT_BAKERY, DEPT_CLEANING, DEPT_ALCOHOL]
select_from_departments(products , product_id, dep_id)

#TEST2
print("\n------------------------------------------------------------------------------------------")
print("TEST CASE 2\n")
product_id = [92, 71, 28, 68, 6, 83, 44, 3]
dep_id = [DEPT_WORLD, DEPT_ALCOHOL]
select_from_departments(products , product_id, dep_id)

#TEST3
print("\n------------------------------------------------------------------------------------------")
print("TEST CASE 3\n")
product_id = [97, 54, 30, 48, 16, 5, 70, 39]
dep_id = [DEPT_DRINKS, DEPT_BABIES, DEPT_PETS]
select_from_departments(products , product_id, dep_id)


------------------------------------------------------------------------------------------
EXAMPLE

Input Products:
22 Fresh Breath Oral Rinse Mild Mint , department number: 11
26 Fancy Feast Trout Feast Flaked Wet Cat Food , department number: 8
45 European Cucumber , department number: 4
54 24/7 Performance Cat Litter , department number: 8
57 Flat Toothpicks , department number: 17
71 Ultra 7 Inch Polypropylene Traditional Plates , department number: 17
111 Fabric Softener, Geranium Scent , department number: 17
112 Hot Tomatillo Salsa , department number: 13

Selected Products:
57 Flat Toothpicks , department number: 17
71 Ultra 7 Inch Polypropylene Traditional Plates , department number: 17
111 Fabric Softener, Geranium Scent , department number: 17

------------------------------------------------------------------------------------------
TEST CASE 1

Input Products:
42 Biotin 1000 mcg , department number: 11
15 Overnight Diapers Size 6 , department number: 18
91 Kind Prenatal O

## 2.2. Read and filter transactions

In [15]:
def select_from_departments_noprints(df , prod_ids, department_ids):
    return_list = []
    
    for product_id in prod_ids:
        dep_id = df.loc[product_id].department_id        
        if dep_id == department_ids:
            return_list.append(product_id)

    return return_list

In [16]:
with gzip.open(INPUT_TRANSACTIONS, "rt") as inputfile:

    reader = csv.reader(inputfile, delimiter=",")

    transactions = [] 
    transactions_read = 0

    for row in reader:
        items = [int(x) for x in row]
        
        if len(select_from_departments_noprints(products , items, DEPT_CLEANING))!=0:
            transactions.append(items)
            
        #OTHER WAY TO SOLVE THE EXERCISE   
        #transaction_appended = False
        #for item in items:
            #if (DEPT_CLEANING == products.loc[item].department_id) and (transaction_appended == False):
                #transactions.append(items)
                #transaction_appended = True
    
        transactions_read += 1
                    
        if transactions_read % 1000 == 0:
            print("Transactions read:", transactions_read, ", Transactions stored:", len(transactions))

        if len(transactions) >= 5000:
            print("\nEXECUTION FINISHED WITH THE FOLLOWING RESULTS:")
            print("Transactions read:", transactions_read, ", Transactions stored:", len(transactions))
            break


Transactions read: 1000 , Transactions stored: 158
Transactions read: 2000 , Transactions stored: 311
Transactions read: 3000 , Transactions stored: 460
Transactions read: 4000 , Transactions stored: 598
Transactions read: 5000 , Transactions stored: 745
Transactions read: 6000 , Transactions stored: 902
Transactions read: 7000 , Transactions stored: 1067
Transactions read: 8000 , Transactions stored: 1206
Transactions read: 9000 , Transactions stored: 1373
Transactions read: 10000 , Transactions stored: 1515
Transactions read: 11000 , Transactions stored: 1670
Transactions read: 12000 , Transactions stored: 1807
Transactions read: 13000 , Transactions stored: 1951
Transactions read: 14000 , Transactions stored: 2102
Transactions read: 15000 , Transactions stored: 2245
Transactions read: 16000 , Transactions stored: 2384
Transactions read: 17000 , Transactions stored: 2543
Transactions read: 18000 , Transactions stored: 2692
Transactions read: 19000 , Transactions stored: 2840
Transact

## 2.3. Extract association rules and comment on them (DEPT_CLEANING)

In [17]:
results = list(apriori(transactions, min_support=0.01, min_confidence=0.2, min_lift=1))
print_apyori_output(results)

Rules involving itemset [13176, 5876]
[5876] => [13176] (support=0.0104, confidence=0.37, lift=3.25)

Rules involving itemset [47209, 5876]
[5876] => [47209] (support=0.0100, confidence=0.36, lift=5.80)

Rules involving itemset [13176, 8021]
[8021] => [13176] (support=0.0114, confidence=0.21, lift=1.84)

Rules involving itemset [13176, 21137]
[21137] => [13176] (support=0.0186, confidence=0.30, lift=2.67)

Rules involving itemset [13176, 21903]
[21903] => [13176] (support=0.0136, confidence=0.25, lift=2.17)

Rules involving itemset [13176, 27966]
[27966] => [13176] (support=0.0158, confidence=0.39, lift=3.40)

Rules involving itemset [13176, 47209]
[47209] => [13176] (support=0.0222, confidence=0.36, lift=3.15)

Rules involving itemset [24852, 16797]
[16797] => [24852] (support=0.0138, confidence=0.31, lift=2.07)

Rules involving itemset [21137, 47209]
[21137] => [47209] (support=0.0126, confidence=0.21, lift=3.35)
[47209] => [21137] (support=0.0126, confidence=0.20, lift=3.35)

Rules 

Based on the association rules, there are some 2 main options that pop up as good ways for the app to improve:

1. **Promotional Packages**: The app can create offers based on the association rules. For instance, they could promote items like [5876] along with [13176] since they have a lift of 3.25, indicating a strong association. This can be presented as a "Often Bought Together" or "Special Offer" to encourage customers to buy both items.

2. **Recommendations**: Implement a recommendations algorithm that suggests items based on the association rules. For instance, if a customer adds [5876] to their cart, the app can recommend [13176] and vice versa.


## 2.4. Extract association rules and comment on them (other departments)

In [18]:
def select_from_departments_noprints2(df , prod_ids, department_ids):
    return_list = []
    
    for product_id in prod_ids:
        dep_id = df.loc[product_id].department_id        
        if dep_id in department_ids:
            return_list.append(product_id)

    return return_list

In [19]:
with gzip.open(INPUT_TRANSACTIONS, "rt") as inputfile:

    reader = csv.reader(inputfile, delimiter=",")
    dept_ids = [DEPT_DRINKS,DEPT_ALCOHOL]

    transactions = [] 
    transactions_read = 0

    for row in reader:
        items = [int(x) for x in row]
        
        if len(select_from_departments_noprints2(products , items, dept_ids))!=0:
            transactions.append(items)
            
        transactions_read += 1
                    
        if transactions_read % 1000 == 0:
            print("Transactions read:", transactions_read, ", Transactions stored:", len(transactions))

        if len(transactions) >= 5000:
            print("\nEXECUTION FINISHED WITH THE FOLLOWING RESULTS:")
            print("Transactions read:", transactions_read, ", Transactions stored:", len(transactions))
            break
            
results = list(apriori(transactions, min_support=0.01, min_confidence=0.2, min_lift=1))
print_apyori_output(results)

Transactions read: 1000 , Transactions stored: 453
Transactions read: 2000 , Transactions stored: 901
Transactions read: 3000 , Transactions stored: 1395
Transactions read: 4000 , Transactions stored: 1872
Transactions read: 5000 , Transactions stored: 2326
Transactions read: 6000 , Transactions stored: 2785
Transactions read: 7000 , Transactions stored: 3256
Transactions read: 8000 , Transactions stored: 3743
Transactions read: 9000 , Transactions stored: 4190
Transactions read: 10000 , Transactions stored: 4667

EXECUTION FINISHED WITH THE FOLLOWING RESULTS:
Transactions read: 10644 , Transactions stored: 5000
Rules involving itemset [13176, 21137]
[21137] => [13176] (support=0.0196, confidence=0.25, lift=2.00)

Rules involving itemset [13176, 21903]
[21903] => [13176] (support=0.0184, confidence=0.25, lift=2.02)

Rules involving itemset [13176, 22935]
[22935] => [13176] (support=0.0100, confidence=0.30, lift=2.36)

Rules involving itemset [13176, 27966]
[27966] => [13176] (support=0

Having looked at the results of the association rules, we come across many of the things we commented for the previous results. Customers purchasing item [27966] are 33% likely to buy item [13176], which with a lift of 2.63 suggests a strong association between these items.  Rules such as this one can be used to make informed decisions about product placements, recommendations, and marketing strategies. 

Another example of promotions for items that are strongly associated  could be [22935] and [13176]. 

Additionally, using the associations we could improve the recommendation system to suggest related items to customers based on these associations, enhancing the customer experience and increasing sales. For instance, suggesting a certain product when another one has been added to the cart, as we mentioned above, or recomending specific products, based on previous sales.

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>